<h3>Módulo para aplicação de algoritmos em base dividida em treino e teste</h3>
Importação das bibliotecas necessárias

In [ ]:
"""
EDUARDO BISPO FELIZARDO | 10400894
GUSTAVO AUGUSTO ALVES PIVATTO | 10401400
JEFFERSON DA CONCEICAO DANTAS | 10401327
MARINA MIKI SINZATO | 10401880 

Objetivo do Notebook
Este notebook tem como foco a geração de previsões utilizando os modelos treinados anteriormente e a 
análise comparativa dos resultados obtidos, com foco na avaliação de desempenho dos modelos aplicados.

Etapas Executadas
Importação de bibliotecas
São carregadas bibliotecas como pandas, statsmodels, xgboost, pmdarima.

Leitura dos dados
O notebook carrega a base atualizada de vendas (Base_Atualizada.csv)
Também importa um arquivo com os resultados de MAPE já calculados anteriormente (resultados_mape.csv).

Previsões com os modelos
Com base nos modelos treinados no notebook anterior, são feitas previsões futuras para a demanda de café.

30/03/2025,EDUARDO BISPO FELIZARDO,Importação de bibliotecas para visualização e modelagem.
30/03/2025,GUSTAVO AUGUSTO ALVES PIVATTO,Leitura da base de dados e dos resultados de MAPE anteriores.
30/03/2025,JEFFERSON DA CONCEICAO DANTAS,Geração de previsões com os modelos treinados.
30/03/2025,MARINA MIKI SINZATO,Comparação dos resultados finais e análise de desempenho dos modelos.

""" 

In [223]:
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from pmdarima import auto_arima

import warnings

<b>Carregamento de arquivos</b>
<br>
Define o caminho do arquivo que contém os dados trataods, que é gerado após rodar o arquivo 2_tratamento_dados.ipynb

In [224]:
caminho_arquivo_base = Path().resolve()  / "Base de Dados/Base_Atualizada.csv"
caminho_resultados_mape = Path().resolve() / "Bases Geradas/resultados_mape.csv"

In [225]:
df_base = pd.read_csv(caminho_arquivo_base, sep=";")
df_resultados = pd.read_csv(caminho_resultados_mape)

In [226]:
df_base['date'] = pd.to_datetime(df_base['date'], format='%d/%m/%Y')
df_base = df_base.sort_values(by='date')

In [227]:
df_base

,date,cash_type,money,coffee_name
0,2024-03-01,card,38.70,Latte
9,2024-03-01,card,33.80,Americano with Milk
8,2024-03-01,card,38.70,Cocoa
7,2024-03-01,card,33.80,Americano with Milk
6,2024-03-01,card,38.70,Hot Chocolate
...,...,...,...,...
3631,2025-03-23,card,35.76,Cappuccino
3632,2025-03-23,card,35.76,Cocoa
3633,2025-03-23,card,35.76,Cocoa
3831,2025-03-23,card,25.96,Americano


<b>Criação de funções para previsão.</b>

In [228]:
def forecast_media_movel(df_cafe, forecast_size):
    df = df_cafe.copy()
    df['media_movel'] = df['sales'].rolling(window=12).mean()

    last_date = df['date'].max()
    
    forecast_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=forecast_size, freq='M')
    forecast_dates = pd.to_datetime([f"{date.year}-{date.month:02d}-01" for date in forecast_dates])

    for i in range(forecast_size):
        valor_previsto = df['media_movel'].iloc[-12:].mean()
        nova_linha = pd.DataFrame({
            'date': forecast_dates[i],
            'sales': [valor_previsto]
        })
        df = pd.concat([df, nova_linha], ignore_index=True)
        df['media_movel'] = df['sales'].ewm(span=12, adjust=False).mean()

    return df

In [229]:
def get_parametros(coffee_name):
    params_row = df_resultados[
        (df_resultados['coffee_name'] == coffee_name) & 
        (df_resultados['modelo'] == 'holtwinters')
    ]

    if not params_row.empty:
        trend = params_row['trend'].values[0]
        seasonal = params_row['seasonal'].values[0]
        seasonal_periods = params_row['seasonal_periods'].values[0]
        return trend, seasonal, seasonal_periods
    else:
        return None, 'add', 6 

In [230]:
def forecast_holtwinters(df_cafe, forecast_size):
    df = df_cafe.copy()

    df['sales'] = df['sales'].apply(lambda x: x if x > 0 else 0.01)

    coffee_name = df['coffee_name'].iloc[0]

    trend, seasonal, seasonal_periods = get_parametros(coffee_name)

    # Ajuste do modelo HoltWinters com os parametros lidos
    model = ExponentialSmoothing(df['sales'], 
                                 trend=trend,    
                                 seasonal=seasonal, 
                                 use_boxcox=True,
                                 seasonal_periods=seasonal_periods)  
    fit_model = model.fit()

    last_date = df['date'].max()
    forecast_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=forecast_size, freq='M')
    forecast_dates = pd.to_datetime([f"{date.year}-{date.month:02d}-01" for date in forecast_dates])

    holtwinters_forecast = fit_model.forecast(forecast_size)

    result = pd.DataFrame({
        'date': forecast_dates,
        'sales': holtwinters_forecast  
    })

    df = pd.concat([df, result], ignore_index=True)

    return df

In [231]:
def forecast_arima(df_cafe, forecast_size):
    df = df_cafe.copy()
    
    # Ajuste do modelo auto_arima
    model_auto_arima = auto_arima(df['sales'],
                                  start_p=0, start_q=0, start_d=0,
                                  max_p=12, max_d=2, max_q=12,
                                  start_P=0, start_Q=0, D=1,
                                  max_P=1, max_Q=1,
                                  seasonal=True, m=4,
                                  trace=False,
                                  suppress_warnings=True,
                                  stepwise=True,
                                  max_iter=50)
    
    forecast = model_auto_arima.predict(n_periods=forecast_size)
    
    forecast_dates = pd.date_range(df['date'].max() + pd.DateOffset(months=1), periods=forecast_size, freq='M')
    forecast_dates = pd.to_datetime([f"{date.year}-{date.month:02d}-01" for date in forecast_dates])

    result = pd.DataFrame({
        'date': forecast_dates,
        'sales': forecast
    })
    
    df = pd.concat([df, result], ignore_index=True)

    return df

In [232]:
def forecast_xgboost_lags(df_cafe, lags=6, forecast_size=12):
    df = df_cafe.copy()
    df['ANO'] = df['date'].dt.year
    df['MES'] = df['date'].dt.month
    
    for lag in range(1, lags + 1):
        df[f'lag_{lag}'] = df['sales'].shift(lag)
    
    df = df.dropna()
    
    features_cols = ['MES', 'ANO'] + [f'lag_{lag}' for lag in range(1, lags + 1)]
    X = df[features_cols].copy()
    y = df['sales'].copy()
    
    #Ajuste do modelo XGBoost
    modelo_xgb = XGBRegressor(
        random_state=42,
        n_estimators=600,
        learning_rate=0.1,
        subsample=0.85,
        max_depth=4,
        reg_alpha=0.01,
        reg_lambda=1,
        objective='reg:squarederror'
    )
    
    modelo_xgb.fit(X, y)
    
    last_date = df['date'].max()
    forecast_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=forecast_size, freq='M')
    forecast_dates = pd.to_datetime([f"{date.year}-{date.month:02d}-01" for date in forecast_dates])
    
    forecast_input = X.iloc[-1:].copy()
    forecast_values = []
    
    for i in range(forecast_size):
        prediction = modelo_xgb.predict(forecast_input)[0]
        forecast_values.append(prediction)
        
        forecast_input = forecast_input.copy()
        forecast_input[f'lag_1'] = prediction
        for lag in range(2, lags + 1):
            forecast_input[f'lag_{lag}'] = forecast_input[f'lag_{lag-1}']
    
    result = pd.DataFrame({
        'date': forecast_dates,
        'sales': forecast_values
    })
    
    df = pd.concat([df, result], ignore_index=True)
    return df


In [233]:
model_functions = {
    "holtwinters": forecast_holtwinters,
    "media_movel": forecast_media_movel,
    "xgboost": forecast_xgboost_lags,
    "auto_arima": forecast_arima
}

In [234]:
df_melhor_modelo = (df_resultados
                    .loc[df_resultados.groupby(['coffee_name'])['MAPE'].idxmin()])

<b>Criação de Loop para iterar pelos tipos de café</b><br>

In [235]:
warnings.filterwarnings("ignore")

resultados = []  

cafes_validos = df_base['coffee_name'].value_counts()
cafes_validos = cafes_validos[cafes_validos >= 24].index

df_base_filtrado = df_base[df_base['coffee_name'].isin(cafes_validos)]

for cafe in cafes_validos:
    df_cafe = df_base_filtrado[df_base_filtrado['coffee_name'] == cafe].copy()
    df_cafe = df_cafe.groupby(df_cafe["date"].dt.to_period("M")).size().reset_index(name="sales")
    df_cafe["date"] = df_cafe["date"].dt.to_timestamp()
    df_cafe["coffee_name"] = cafe  
    
    modelo_values = df_melhor_modelo[df_melhor_modelo['coffee_name'] == cafe]['modelo'].values
    
    if len(modelo_values) > 0:
        modelo = modelo_values[0]
        func = model_functions.get(modelo)
        
        if func:
            print(f"Executando {modelo} para Café: {cafe}")
            previsao = func(df_cafe, forecast_size=12)
            
            previsao["coffee_name"] = cafe
            previsao["modelo"] = modelo
            
            resultados.append(previsao)
        else:
            print(f"Modelo {modelo} não encontrado para Café: {cafe}")
    else:
        print(f"Nenhum modelo encontrado para Café: {cafe}")

df_previsao_final = pd.concat(resultados, ignore_index=True)
df_previsao_final['sales'] = df_previsao_final['sales'].round(0)

Executando xgboost para Café: Americano with Milk
Executando media_movel para Café: Latte
Executando auto_arima para Café: Americano
Executando auto_arima para Café: Cappuccino
Executando media_movel para Café: Cortado
Executando auto_arima para Café: Hot Chocolate
Executando xgboost para Café: Cocoa
Executando holtwinters para Café: Espresso


In [236]:
nova_ordem_colunas = ['date', 'coffee_name', 'sales']

df_previsao_final = df_previsao_final[nova_ordem_colunas]

In [237]:
df_previsao_final.to_csv("Resultado/previsao_cafes.csv", index=False)